# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. System consists of two M/M/1 queues put one after another. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
from itertools import zip_longest
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_seconds):
    secs = total_seconds % 60
    mins = int((total_seconds / 60) % 60)
    hours = int(total_seconds //60 //60)
    return f" {hours:02}:{mins:02}:{secs:04.1f}"
        
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

In [3]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

LAST_EXIT = 0

#RANDOM_SEED = 971
#random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []


break_count = 0



We modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [7]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue. \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue. \t at {format_time(self.finish_operator)}")
        
        if self.env.now >=60*60*8:
            #Shop is closed
            print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
            print(f"-- shop is closed")
        else:
            with expert.request() as exp_req: 
                enter_e_q = self.env.now
                self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
                results = yield exp_req | env.timeout(self.patience)
                self.wait_exp=self.env.now-self.finish_operator
                second_queue_w_times.append(self.wait_exp) 
                exit_e_q = self.env.now
                global expert_queue_log
                expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

                if exp_req not in results:
                    #Leave queue, regened
                    print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
                    print(f"-- after waiting {format_time(self.wait_exp)}")

                else:
                # We got to the expert
                    print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                    if self.wait_exp:
                        print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                    yield self.env.process(self.ask_question_expert())
                    self.finish_expert=self.env.now
                    print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        # print("%s ask_question-op at  %g , service_opr= %g" % (self.id, self.env.now, service_opr))
        yield self.env.timeout(service_opr)
        # print("%s ask_question-op at  %g finished " % (self.id, self.env.now))

        first_service_times.append(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        # print("%s ask_question-exp at  %g , service_expr= %g" % (self.id, self.env.now,service_exp))
        yield self.env.timeout(service_exp)
        second_service_times.append(service_exp)   # Why is this after yield

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [8]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        if env.now >= 60*60*8:
            break
        time_between_next_customer = random.expovariate(1.0 / INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)
        if env.now + time_between_next_customer >= 60*60*8:
            break
        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while True:
        time_to_break = random.expovariate(1/60/60)
        if env.now + time_to_break >= 60*60*8:
            break
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1


Here, we define our processes and resources and start running the simulation environment!

In [9]:
env = simpy.Environment()
CUSTOMER_NUM=CALLNUM2
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00.0
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00.0
Cust    2 entered the 		 FRONT Queue. 	 at  00:00:06.1
Cust    1 entered the 		 EXPERT Queue. 	 at  00:00:06.7
Cust    2 started talking with 	 FRONT Desk 	 at  00:00:06.7
Cust    2 waited in 		 FRONT Queue 	 for 00:00:00.6
Cust    1 started talking with 	 EXPERT 	 at  00:00:06.7
Cust    1 exited 		 NORMALLY 	 at  00:00:07.8
Cust    2 entered the 		 EXPERT Queue. 	 at  00:00:14.0
Cust    2 started talking with 	 EXPERT 	 at  00:00:14.0
Cust    3 entered the 		 FRONT Queue. 	 at  00:00:18.1
Cust    3 started talking with 	 FRONT Desk 	 at  00:00:18.1
Cust    4 entered the 		 FRONT Queue. 	 at  00:00:21.4
Cust    3 entered the 		 EXPERT Queue. 	 at  00:00:24.5
Cust    4 started talking with 	 FRONT Desk 	 at  00:00:24.5
Cust    4 waited in 		 FRONT Queue 	 for 00:00:03.1
Cust    4 entered the 		 EXPERT Queue. 	 at  00:00:30.7
Cust    5 entered the 		 FRONT Queue. 	 at  00:0

Cust  219 entered the 		 FRONT Queue. 	 at  00:57:30.1
Cust  219 started talking with 	 FRONT Desk 	 at  00:57:30.1
Cust  217 exited 		 NORMALLY 	 at  00:57:30.1
Cust  218 started talking with 	 EXPERT 	 at  00:57:30.1
Cust  218 waited in 		 EXPERT Queue 	 for 00:00:05.2
Cust  219 entered the 		 EXPERT Queue. 	 at  00:57:38.7
Cust  218 exited 		 NORMALLY 	 at  00:57:39.8
Cust  219 started talking with 	 EXPERT 	 at  00:57:39.8
Cust  219 waited in 		 EXPERT Queue 	 for 00:00:01.1
Cust  219 exited 		 NORMALLY 	 at  00:57:44.4
Cust  220 entered the 		 FRONT Queue. 	 at  00:57:47.8
Cust  220 started talking with 	 FRONT Desk 	 at  00:57:47.8
Cust  220 entered the 		 EXPERT Queue. 	 at  00:57:54.5
Cust  220 started talking with 	 EXPERT 	 at  00:57:54.5
Cust  221 entered the 		 FRONT Queue. 	 at  00:57:54.9
Cust  221 started talking with 	 FRONT Desk 	 at  00:57:54.9
Cust  221 entered the 		 EXPERT Queue. 	 at  00:58:01.2
Cust  220 exited 		 NORMALLY 	 at  00:58:10.0
Cust  221 started talki

Cust  442 exited 		 NORMALLY 	 at  01:52:21.4
Cust  444 entered the 		 EXPERT Queue. 	 at  01:52:23.8
Cust  445 started talking with 	 FRONT Desk 	 at  01:52:23.8
Cust  445 waited in 		 FRONT Queue 	 for 00:00:05.5
Cust  444 started talking with 	 EXPERT 	 at  01:52:23.8
Cust  446 entered the 		 FRONT Queue. 	 at  01:52:25.4
Cust  445 entered the 		 EXPERT Queue. 	 at  01:52:29.6
Cust  446 started talking with 	 FRONT Desk 	 at  01:52:29.6
Cust  446 waited in 		 FRONT Queue 	 for 00:00:04.3
Cust  447 entered the 		 FRONT Queue. 	 at  01:52:30.8
Cust  448 entered the 		 FRONT Queue. 	 at  01:52:32.0
Cust  446 entered the 		 EXPERT Queue. 	 at  01:52:36.1
Cust  447 started talking with 	 FRONT Desk 	 at  01:52:36.1
Cust  447 waited in 		 FRONT Queue 	 for 00:00:05.3
Cust  446 exited 		 SUDDENLY 	 at  01:52:36.9
-- after waiting  00:00:00.8
Cust  447 entered the 		 EXPERT Queue. 	 at  01:52:44.0
Cust  448 started talking with 	 FRONT Desk 	 at  01:52:44.0
Cust  448 waited in 		 FRONT Queu

Cust  634 started talking with 	 FRONT Desk 	 at  02:37:28.0
Cust  634 waited in 		 FRONT Queue 	 for 00:00:02.0
Cust  631 exited 		 NORMALLY 	 at  02:37:29.8
Cust  632 started talking with 	 EXPERT 	 at  02:37:29.8
Cust  632 waited in 		 EXPERT Queue 	 for 00:00:12.1
Cust  632 exited 		 NORMALLY 	 at  02:37:29.8
Cust  633 started talking with 	 EXPERT 	 at  02:37:29.8
Cust  633 waited in 		 EXPERT Queue 	 for 00:00:01.8
Cust  634 entered the 		 EXPERT Queue. 	 at  02:37:35.5
Cust  633 exited 		 NORMALLY 	 at  02:37:46.7
Cust  634 started talking with 	 EXPERT 	 at  02:37:46.7
Cust  634 waited in 		 EXPERT Queue 	 for 00:00:11.3
Cust  635 entered the 		 FRONT Queue. 	 at  02:37:54.0
Cust  635 started talking with 	 FRONT Desk 	 at  02:37:54.0
Cust  635 entered the 		 EXPERT Queue. 	 at  02:38:00.8
Cust  635 exited 		 SUDDENLY 	 at  02:38:08.0
-- after waiting  00:00:07.2
Cust  636 entered the 		 FRONT Queue. 	 at  02:38:27.3
Cust  636 started talking with 	 FRONT Desk 	 at  02:38:27.3


Cust  841 started talking with 	 FRONT Desk 	 at  03:27:06.5
Cust  841 waited in 		 FRONT Queue 	 for 00:00:06.1
Cust  840 started talking with 	 EXPERT 	 at  03:27:06.5
Cust  842 entered the 		 FRONT Queue. 	 at  03:27:07.5
Cust  840 exited 		 NORMALLY 	 at  03:27:09.1
Cust  841 entered the 		 EXPERT Queue. 	 at  03:27:12.5
Cust  842 started talking with 	 FRONT Desk 	 at  03:27:12.5
Cust  842 waited in 		 FRONT Queue 	 for 00:00:05.0
Cust  841 started talking with 	 EXPERT 	 at  03:27:12.5
Cust  841 exited 		 NORMALLY 	 at  03:27:12.7
Cust  843 entered the 		 FRONT Queue. 	 at  03:27:16.0
Cust  842 entered the 		 EXPERT Queue. 	 at  03:27:19.5
Cust  843 started talking with 	 FRONT Desk 	 at  03:27:19.5
Cust  843 waited in 		 FRONT Queue 	 for 00:00:03.5
Cust  842 started talking with 	 EXPERT 	 at  03:27:19.5
Cust  842 exited 		 NORMALLY 	 at  03:27:19.8
Cust  844 entered the 		 FRONT Queue. 	 at  03:27:23.2
Cust  843 entered the 		 EXPERT Queue. 	 at  03:27:27.4
Cust  844 started t

Cust 1028 entered the 		 EXPERT Queue. 	 at  04:13:57.8
Cust 1027 exited 		 NORMALLY 	 at  04:14:04.4
Cust 1028 started talking with 	 EXPERT 	 at  04:14:04.4
Cust 1028 waited in 		 EXPERT Queue 	 for 00:00:06.6
Cust 1028 exited 		 NORMALLY 	 at  04:14:16.9
Cust 1029 entered the 		 FRONT Queue. 	 at  04:14:18.8
Cust 1029 started talking with 	 FRONT Desk 	 at  04:14:18.8
Cust 1030 entered the 		 FRONT Queue. 	 at  04:14:23.8
Cust 1029 entered the 		 EXPERT Queue. 	 at  04:14:27.7
Cust 1030 started talking with 	 FRONT Desk 	 at  04:14:27.7
Cust 1030 waited in 		 FRONT Queue 	 for 00:00:03.8
Cust 1029 started talking with 	 EXPERT 	 at  04:14:27.7
Cust 1031 entered the 		 FRONT Queue. 	 at  04:14:28.6
Cust 1032 entered the 		 FRONT Queue. 	 at  04:14:30.1
Cust 1030 entered the 		 EXPERT Queue. 	 at  04:14:33.1
Cust 1031 started talking with 	 FRONT Desk 	 at  04:14:33.1
Cust 1031 waited in 		 FRONT Queue 	 for 00:00:04.5
Cust 1031 entered the 		 EXPERT Queue. 	 at  04:14:41.5
Cust 1032 

Cust 1218 started talking with 	 EXPERT 	 at  04:53:50.4
Cust 1218 waited in 		 EXPERT Queue 	 for 00:00:05.8
Cust 1219 entered the 		 FRONT Queue. 	 at  04:53:52.3
Cust 1219 started talking with 	 FRONT Desk 	 at  04:53:52.3
Cust 1218 exited 		 NORMALLY 	 at  04:53:58.6
Cust 1219 entered the 		 EXPERT Queue. 	 at  04:53:60.0
Cust 1219 started talking with 	 EXPERT 	 at  04:53:60.0
Cust 1220 entered the 		 FRONT Queue. 	 at  04:54:00.8
Cust 1220 started talking with 	 FRONT Desk 	 at  04:54:00.8
Cust 1221 entered the 		 FRONT Queue. 	 at  04:54:04.0
Cust 1220 entered the 		 EXPERT Queue. 	 at  04:54:07.4
Cust 1221 started talking with 	 FRONT Desk 	 at  04:54:07.4
Cust 1221 waited in 		 FRONT Queue 	 for 00:00:03.4
Cust 1221 entered the 		 EXPERT Queue. 	 at  04:54:13.6
Cust 1219 exited 		 NORMALLY 	 at  04:54:15.8
Cust 1220 started talking with 	 EXPERT 	 at  04:54:15.8
Cust 1220 waited in 		 EXPERT Queue 	 for 00:00:08.4
Cust 1220 exited 		 NORMALLY 	 at  04:54:17.5
Cust 1221 started

Cust 1422 started talking with 	 FRONT Desk 	 at  05:44:44.9
Cust 1422 waited in 		 FRONT Queue 	 for 00:00:06.0
Cust 1421 started talking with 	 EXPERT 	 at  05:44:44.9
Cust 1422 entered the 		 EXPERT Queue. 	 at  05:44:51.8
Cust 1423 started talking with 	 FRONT Desk 	 at  05:44:51.8
Cust 1423 waited in 		 FRONT Queue 	 for 00:00:08.7
Cust 1421 exited 		 NORMALLY 	 at  05:44:54.4
Cust 1422 started talking with 	 EXPERT 	 at  05:44:54.4
Cust 1422 waited in 		 EXPERT Queue 	 for 00:00:02.6
Cust 1423 entered the 		 EXPERT Queue. 	 at  05:44:57.1
Cust 1424 entered the 		 FRONT Queue. 	 at  05:44:57.6
Cust 1424 started talking with 	 FRONT Desk 	 at  05:44:57.6
Cust 1424 entered the 		 EXPERT Queue. 	 at  05:45:05.0
Cust 1423 exited 		 SUDDENLY 	 at  05:45:08.8
-- after waiting  00:00:11.7
Cust 1422 exited 		 NORMALLY 	 at  05:45:10.4
Cust 1424 started talking with 	 EXPERT 	 at  05:45:10.4
Cust 1424 waited in 		 EXPERT Queue 	 for 00:00:05.3
Cust 1424 exited 		 NORMALLY 	 at  05:45:16.0


Cust 1597 exited 		 SUDDENLY 	 at  06:25:55.4
-- after waiting  00:00:04.6
Cust 1598 entered the 		 EXPERT Queue. 	 at  06:25:57.2
Cust 1596 exited 		 NORMALLY 	 at  06:26:01.0
Cust 1598 started talking with 	 EXPERT 	 at  06:26:01.0
Cust 1598 waited in 		 EXPERT Queue 	 for 00:00:03.8
Cust 1598 exited 		 NORMALLY 	 at  06:26:01.0
Cust 1599 entered the 		 FRONT Queue. 	 at  06:26:09.5
Cust 1599 started talking with 	 FRONT Desk 	 at  06:26:09.5
Cust 1600 entered the 		 FRONT Queue. 	 at  06:26:17.9
Cust 1599 entered the 		 EXPERT Queue. 	 at  06:26:18.8
Cust 1600 started talking with 	 FRONT Desk 	 at  06:26:18.8
Cust 1600 waited in 		 FRONT Queue 	 for 00:00:00.9
Cust 1599 started talking with 	 EXPERT 	 at  06:26:18.8
Cust 1600 entered the 		 EXPERT Queue. 	 at  06:26:26.0
Cust 1601 entered the 		 FRONT Queue. 	 at  06:26:32.2
Cust 1601 started talking with 	 FRONT Desk 	 at  06:26:32.2
Cust 1599 exited 		 NORMALLY 	 at  06:26:32.5
Cust 1600 started talking with 	 EXPERT 	 at  06:26:

Cust 1770 started talking with 	 EXPERT 	 at  07:13:30.5
Cust 1771 entered the 		 EXPERT Queue. 	 at  07:13:40.7
Cust 1771 exited 		 SUDDENLY 	 at  07:13:49.7
-- after waiting  00:00:09.0
Cust 1770 exited 		 NORMALLY 	 at  07:13:49.9
Cust 1772 entered the 		 FRONT Queue. 	 at  07:14:19.2
Cust 1772 started talking with 	 FRONT Desk 	 at  07:14:19.2
Cust 1772 entered the 		 EXPERT Queue. 	 at  07:14:27.0
Cust 1772 started talking with 	 EXPERT 	 at  07:14:27.0
Cust 1772 exited 		 NORMALLY 	 at  07:14:40.2
Cust 1773 entered the 		 FRONT Queue. 	 at  07:15:00.6
Cust 1773 started talking with 	 FRONT Desk 	 at  07:15:00.6
Cust 1773 entered the 		 EXPERT Queue. 	 at  07:15:08.0
Cust 1773 started talking with 	 EXPERT 	 at  07:15:08.0
Cust 1773 exited 		 NORMALLY 	 at  07:15:09.5
Cust 1774 entered the 		 FRONT Queue. 	 at  07:15:39.4
Cust 1774 started talking with 	 FRONT Desk 	 at  07:15:39.4
Cust 1775 entered the 		 FRONT Queue. 	 at  07:15:44.2
Cust 1774 entered the 		 EXPERT Queue. 	 at  

Lastly, we print our statistics we gathered from the simulation

In [ ]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys for w, sys in zip(total_waits, total_system)))


print(expert_queue_log)
stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": "NO IDEA YET",
}
print(stats)

In [ ]:
print (len(first_queue_w_times))

In [ ]:
print (len(second_queue_w_times))

In [ ]:
print(format_time(env.now))
print(format_time(LAST_EXIT))